### MLP vs CNN
#### By MMA

### Fully connected MLP with MNIST

In [1]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<class 'numpy.ndarray'>


In [4]:
LEARNING_RATE = 0.01
EPOCHS = 1000
BATCH_SIZE = 64

In [5]:
x = tf.placeholder(dtype=tf.float32, shape=(None, 784), name="input")
y = tf.placeholder(dtype=tf.float32, shape=(None, 10), name="label")

In [6]:
with tf.name_scope(name="Hidden_layer_1"):
    w1 = tf.Variable(tf.truncated_normal([784, 500], 0, 0.1), name="W")
    b1 = tf.Variable(tf.zeros([500]), name="B")

    hidden_layer1_input = tf.matmul(x, w1) + b1
    relu1 = tf.nn.relu(hidden_layer1_input)

In [7]:
with tf.name_scope(name="Hidden_layer_2"):
    w2 = tf.Variable(tf.truncated_normal([500, 500], 0, 0.1), name="W")
    b2 = tf.Variable(tf.zeros([500]), name="B")

    hidden_layer2_input = tf.matmul(relu1, w2) + b2
    relu2 = tf.nn.relu(hidden_layer2_input)

In [8]:
with tf.name_scope("output_layer"):
    w3 = tf.Variable(tf.truncated_normal([500, 10], 0, 0.1), name="W")
    b3 = tf.Variable(tf.zeros([10]), name="B")

    output_layer_input = tf.matmul(relu2, w3) + b3
    output = tf.nn.softmax(output_layer_input)

##### Cross Entropy, Loss Function, Accuaracy, etc.

In [10]:
with tf.name_scope("xent"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output))

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
    accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32), name="accuracy")


In [11]:
tf.summary.scalar("Cross_Entropy", cross_entropy)
tf.summary.scalar("accuracy_train", accuracy_op)

merge = tf.summary.merge_all()
filewriter = tf.summary.FileWriter("./graphs")

##### Session Run 

In [12]:
sess = tf.Session()
filewriter.add_graph(sess.graph)

sess.run(tf.global_variables_initializer())

##### Learning

In [13]:
acc_val = tf.summary.scalar("accuracy_validation", accuracy_op)
for i in range(EPOCHS):
    batch_xs, batch_ys = mnist.train.next_batch(BATCH_SIZE)
    sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
    if i % 50 == 0:
        b = (sess.run(merge, feed_dict={x: batch_xs, y: batch_ys}))
        filewriter.add_summary(b, i)
        
        batch_xs_v, batch_ys_v = mnist.validation.next_batch(BATCH_SIZE)
        b = (sess.run(acc_val, feed_dict={x: batch_xs_v, y: batch_ys_v}))
        filewriter.add_summary(b, i)

In [14]:
print("")
yyy = sess.run(output, feed_dict={x: mnist.test.images})
[accuracy] = sess.run([accuracy_op], feed_dict={output: yyy, y: mnist.test.labels})
print("The accuracy of test:" + str(accuracy))


The accuracy of test:0.6505


In [15]:
sess.close()
filewriter.close()